In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Master Path

path = r'/Users/jagpreetbatth/Desktop/Career Foundry/Python/11-2021 Instacart Basket Analysis'

In [3]:
# Verify Path

path

'/Users/jagpreetbatth/Desktop/Career Foundry/Python/11-2021 Instacart Basket Analysis'

In [4]:
# Import task downloaded customer.csv dataset

df_cust = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'customers.csv'))

In [5]:
# Check shape

df_cust.shape

(206209, 10)

In [6]:
# Verify Correct Import 

df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# Wrangle Imported Customer Data

In [7]:
# Note 'Surnam' column is missing an 'e', should be 'Surname'

df_cust.rename(columns = {'Surnam' : 'Surname'}, inplace = True)

In [8]:
# Note 'n-dependants' column is misspelled and does not require the 'n-'

df_cust.rename(columns = {'n_dependants' : 'dependents'}, inplace = True)

In [9]:
# Note 'fam_status' column is not clear cut, should be marital_status

df_cust.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

# Drop column function not needed, all data pertinent

In [10]:
# Verify Changes

df_cust.head()

,user_id,First Name,Surname,Gender,STATE,Age,date_joined,dependents,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [11]:
# Data type summary

df_cust.dtypes

user_id            int64
First Name        object
Surname           object
Gender            object
STATE             object
Age                int64
date_joined       object
dependents         int64
marital_status    object
income             int64
dtype: object

# Fundamental data quality and consistency checks

In [12]:
# Overall Data Description

df_cust.describe()

,user_id,Age,dependents,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


#### Consistent data with no apprent outliers noted.

In [13]:
# Check for mixed type data entries

for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

First Name


In [14]:
# Change dtype of First Name column to string

df_cust['First Name'] = df_cust['First Name'].astype('str')

In [15]:
# Re-check for Mixed type data entries

for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

#### None noted

In [16]:
# Check for missing values

df_cust.isnull().sum()

user_id           0
First Name        0
Surname           0
Gender            0
STATE             0
Age               0
date_joined       0
dependents        0
marital_status    0
income            0
dtype: int64

#### None noted

In [17]:
# Check for duplicates

df_cust_dups = df_cust[df_cust.duplicated()]

In [18]:
# Verify Results

df_cust_dups

,user_id,First Name,Surname,Gender,STATE,Age,date_joined,dependents,marital_status,income


#### None Noted

# Combine customer data frame with updated InstaCart Data frame

In [19]:
# Import updated InstaCart data frame

ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_added_flags.pkl'))

In [20]:
# Check shape

ords_prods_merge.shape

(32641268, 24)

In [21]:
# Verify Correct Import 

ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_purchase_price,spending_flag,median_days_since_prior_order,order_frequency_flag
0,2398795,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,Soda,...,Mid-range product,Regularly busy,Least busy days,Average orders,11.0,Regular customer,6.372222,Low spender,20.0,Regular customer
1,2398795,1.0,2.0,3.0,7.0,15.0,10258.0,2.0,0.0,Pistachios,...,Low-range product,Regularly busy,Least busy days,Average orders,11.0,Regular customer,6.372222,Low spender,20.0,Regular customer
2,2398795,1.0,2.0,3.0,7.0,15.0,12427.0,3.0,1.0,Original Beef Jerky,...,Low-range product,Regularly busy,Least busy days,Average orders,11.0,Regular customer,6.372222,Low spender,20.0,Regular customer
3,2398795,1.0,2.0,3.0,7.0,15.0,13176.0,4.0,0.0,Bag of Organic Bananas,...,Mid-range product,Regularly busy,Least busy days,Average orders,11.0,Regular customer,6.372222,Low spender,20.0,Regular customer
4,2398795,1.0,2.0,3.0,7.0,15.0,26088.0,5.0,1.0,Aged White Cheddar Popcorn,...,Low-range product,Regularly busy,Least busy days,Average orders,11.0,Regular customer,6.372222,Low spender,20.0,Regular customer


In [ ]:
# Combine dataframes

ords_prods_cust = ords_prods_merge.merge(df_cust, on = 'user_id')

In [ ]:
# Verify Combine Shape

ords_prods_cust.shape

In [42]:
# Verify Combine Columns

ords_prods_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,First Name,Surname,Gender,STATE,Age,date_joined,dependents,marital_status,income
0,2398795,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1.0,2.0,3.0,7.0,15.0,10258.0,2.0,0.0,Pistachios,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,2398795,1.0,2.0,3.0,7.0,15.0,12427.0,3.0,1.0,Original Beef Jerky,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2398795,1.0,2.0,3.0,7.0,15.0,13176.0,4.0,0.0,Bag of Organic Bananas,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,2398795,1.0,2.0,3.0,7.0,15.0,26088.0,5.0,1.0,Aged White Cheddar Popcorn,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


# Export newly merged dataframe as a pickle file

In [44]:
ords_prods_cust.to_pickle(os.path.join(path, 'Data','Prepared Data', 'orders_products_customers_merged.pkl'))